# Add Libraries

In [ ]:
import os
import time
import pandas as pd
import random
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import random


from core import core_global as core
from core.spike_monitor import SpikeMonitor
from core.voltage_monitor import VoltageMonitor
from core.weight_matrix_monitor import WeightMatrixMonitor
from layers.rewiring_connection import RewiringConnection
from models.poisson_pattern_group import PoissonPatternGroup
from utils import utils as utils
from models.mc_lif_group import McLifGroup

# Initialize with an Explore phase results

In [ ]:
## Set the Restuls folder address
exp_name = 'rewiring_ex1'
exp_aspect = 'Exploration_Phase'
simulation_date = "241126_124932"
trial = 0
sim_date = f"{simulation_date}/{trial}"

## Set the model Parameteres
assembly_number = 9
assembly_size = 30
CA3_size = assembly_number*assembly_size
num_branches = 3
cell_num = 100
AcDcell_num = 50

# Run the Ripple Phase

In [ ]:
def main(experiment, sim_date, assemblies,
         assembly_neurons_idc, config, output_directory):
    

    input_params = config["input_parameters"]
    connection_params = config["connection_parameters"]
    neuron_params = config["neuron_parameters"]
    
    ######################### Change of Parameters##################
    config['simulation_time'] = 3000.0
    neuron_params['branch_parameters']['inh_amp'] = -20e-9#  ## dendritic inibition
    neuron_params['branch_parameters']['inh_freq'] = 150          ## dendritic inibition
    
    ### set the 2 Hz frequency for Ripple Phase
    input_params["pattern_delay"] = 0.400
    input_params["pattern_duration"] = 0.100
    neuron_params['branch_parameters']["pattern_delay"] = input_params["pattern_delay"]
    neuron_params['branch_parameters']["pattern_duration"] = input_params["pattern_duration"]
    
    
    print("##############Parameters#############")
    print("Network_size: (CA3, CA1) = (", input_params['num_inputs'],",",config["num_neurons"],")")
    print("Simulation_time:",config['simulation_time']," s")
    print("####################################")
    

    exp_result_name_folder = "Ripple_Phase"

    
    
    ##############################################################

    # Directory for simulation results and log files.
    output_directory = os.path.join("results", experiment,exp_result_name_folder, time.strftime("%y%m%d_%H%M%S"), str(trial),
                                    "data")

    # Initialize the simulation environment.
    core.init(directory=output_directory)

    # Write config file to the output directory.
    utils.write_configuration(os.path.join(output_directory, "..", "initialization.yaml"), config)

    # Set the random seed.
    core.kernel.set_master_seed(config["master_seed"])

    # Create input neurons.
    inp = PoissonPatternGroup(input_params["num_inputs"], input_params["rate"], input_params["rate_bg"],
                              params=input_params)

    # Create the network.
    neuron = McLifGroup(config['num_neurons'], neuron_params["num_branches"], neuron_params)

    # Creat the Connections.
    conn = RewiringConnection(inp, neuron, neuron.branch.syn_current, connection_params)

    # Create some monitors which will record the simulation data.
    for cell_num in range(config['num_neurons']):
        WeightMatrixMonitor(conn, core.kernel.fn(f"cell_recording/cell_num_{cell_num}/weights_sleep", "dat"),cell_id = cell_num ,
                    interval=config["sampling_interval_weights"])
        
        
    
    SpikeMonitor(neuron, core.kernel.fn("output_sleep", "ras"))
    sm_inp = SpikeMonitor(inp, core.kernel.fn("input_sleep", "ras"))
    vm_nrn = [VoltageMonitor(neuron.soma, cell_ind, core.kernel.fn(f"cell_recording/cell_num_{cell_ind}/soma_sleep", "mem"),paste_spikes=True, paste_spikes_from=neuron) for cell_ind in range(config['num_neurons'])]
    vm_br = [[] for cell_ind in range(config['num_neurons'])]
    for cell_ind in range(config['num_neurons']):
        for dend_ind in range(neuron_params["num_branches"]):
            vm_br[cell_ind].append(VoltageMonitor(neuron.branch, (dend_ind, cell_ind), core.kernel.fn(f"cell_recording/cell_num_{cell_ind}/branch_sleep", "mem", dend_ind)))

    # Now simulate the model.
    conn.set_weights(weights_post)
    simulation_time = config["simulation_time"]
    core.kernel.run_chunk(20.0, 0, simulation_time)

    sm_inp.active = True
    for cell_ind in range(config['num_neurons']):
        vm_nrn[cell_ind].active = False
        for vm in vm_br[cell_ind]:
            vm.active = False
            
    core.kernel.run_chunk(simulation_time - 40, 0, simulation_time)

    sm_inp.active = True
    for cell_ind in range(config['num_neurons']):
        vm_nrn[cell_ind].active = True
        for vm in vm_br[cell_ind]:
            vm.active = True

    core.kernel.run_chunk(20.0, 0, simulation_time)


In [ ]:
if __name__ == '__main__':
    import numpy as np

    sim_date = f"{simulation_date}/{trial}"
    master_seed = 13
    
    assemblies = list(range(assembly_number))
    assembly_neurons_idc = [i * assembly_size + np.arange(assembly_size) for i in assemblies]

    input_directory = os.path.join("results", exp_name,exp_aspect, sim_date, "data")

    # Load the configuration file.
    config = utils.load_configuration(os.path.join(
        input_directory, "..", "initialization" + ".yaml"))
    num_branches = config["neuron_parameters"]["num_branches"]
    config["master_seed"] = master_seed
    np.save(os.path.join(input_directory, "assembly_neurons_idc"),
            assembly_neurons_idc)
    # Load the weights.
    weights_post = np.zeros(shape = (num_branches,cell_num,CA3_size))
    recurrent_weights_post = np.zeros(shape = (cell_num,cell_num))
    for cell_ind in range(cell_num):
    
        with open(os.path.join(input_directory, "cell_recording",f"cell_num_{cell_ind}","weights.0.dat"), "rb") as f:
            lines = f.readlines()
        weights_post[:,cell_ind,:] = np.loadtxt(lines[-num_branches-3:-2])

##########################################################
        
    main( exp_name, sim_date, assemblies,assembly_neurons_idc, config, input_directory)
